In [304]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [305]:
url = "https://en.wikipedia.org/wiki/List_of_female_Academy_Award_winners_and_nominees_for_non-gendered_categories"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

#### Raspagem: lista de mulheres nomeadas e ganhadoras do Oscar em categorias em que gêneros competem.

Como alguns elementos ocupam mais que uma linha, não é possível fazer a raspagem iterando entre os múltiplos de cada célula. A solução encontrada foi:

- Para o ano ('year'): tentar converter os 4 primeiros caracteres da célula em inteiro. Se não houver erro, é porque a célula é um ano válido; ele é salvo na variável `new_year`.
- Para o status ('Nominated' ou 'Won'): todas as células de status possuem `<td class="no table-no2" (...)>` para _Nominated_ ou `<td class="yes table-yes2" (...)>` para _Won_. Verifica-se se a célula atual é uma dessas e, se sim, o texto dela, da categoria (título da tabela) e do ano `new_year` é adicionado em suas respectivas listas.


In [306]:
year = []
category = []
status = []
all_tables = soup.find_all(class_='wikitable')

for table in all_tables:
	td = table.find_all('td')
	new_year = 0
	for i in range(0, len(td)):
		try:
			old_year = new_year
			new_year = int(td[i].text[0:4])
			if new_year < 1000:
				new_year = old_year
		except ValueError:
			pass
		if td[i].get('class'):
			status.append(td[i].text[0:-1])
			category.append(td[0].text[18:-1])
			if new_year != 0:
				year.append(new_year)

oscar_women_df = pd.DataFrame({
	'year': year,
	'category': category,
	'status': status
})

#	renomeia as categorias que possuíam sobrescrito no título.
oscar_women_df.category = oscar_women_df.category.replace({
	'Best Sound [note 12]': 'Best Sound Mixing',
	'Best Original Score [note 7]': 'Best Original Score'
})

#	corrige um typo na tabela original.
oscar_women_df.status = oscar_women_df.status.replace({'Nominated}': 'Nominated'})

oscar_women_df


,year,category,status
0,2007,Best Animated Feature,Nominated
1,2011,Best Animated Feature,Nominated
2,2012,Best Animated Feature,Won
3,2013,Best Animated Feature,Won
4,2013,Best Animated Feature,Nominated
...,...,...,...
1418,2017,Best Original Screenplay,Nominated
1419,2017,Best Original Screenplay,Nominated
1420,2018,Best Original Screenplay,Nominated
1421,1917,Best Original Screenplay,Nominated


# Indicados e ganhadores homens.
A raspagem de competidores homens foi feita a partir da [base de dados da Academia de Artes e Ciências Cinematográficas](https://awardsdatabase.oscars.org/). Para automação completa, seria necessária o uso do Selenium. Neste caso, o código fonte foi salvo em um arquivo e a manipulação foi feita em seguida. As seguintes categorias foram selecionadas:

1. Best Picture
2. Best Director
3. Best Original Screenplay
4. Best Adapted Screenplay
5. Best Cinematography
6. Best Costume Design
7. Best Production Design
8. Documentary Feature
9. Best Film Editing
10. Best Makeup and Hairstyling
11. Best International Feature Film
12. Best Visual Effects

### A FAZER:
[ ] Adicionar coluna com status (ganhador ou indicado);
[ ] Pesquisar um jeito mais simples de usar a split com múltiplos delimitadores, sem precisar usar o `.replace()` (linha 14)

In [309]:
def get_oscar_df(file):
	f = open(file)
	soup = BeautifulSoup(f, 'html.parser')
	df = pd.DataFrame({'year': [], 'category': [], 'film': [], 'name': []})

	all_groups = soup.find_all(class_='awards-result-chron result-group group-awardcategory-chron')
	for group in all_groups:
		year = group.find(class_="result-group-title").text.strip()[0:4]
		category = group.find(class_="result-subgroup-header").text.strip()
		all_films = group.find_all(class_="awards-result-nomination awards-result-nomination-actingorsimilar")
		for row in all_films:
			film = row.find(class_="awards-result-film-title").text.strip()

			all_names = row.find(class_="awards-result-nominationstatement").text.replace(' and ', ',').replace(';', ',').split(',')

			all_names = [i for i in all_names if 'Producer' not in i]
			for name in all_names:
				name = name.strip()
				row = pd.DataFrame({'year': [year],
									  'category': [category],
									  'film': [film],
									  'name': [name]
									  })
				df = pd.concat([df, row], ignore_index=True)
				df = df[df.name != 'Jr.']
	return df

In [310]:
picture_men_df = get_oscar_df('oscar-pages/best-picture.html')
picture_men_df

,year,category,film,name
0,1951,BEST MOTION PICTURE,An American in Paris,Arthur Freed
1,1951,BEST MOTION PICTURE,Decision before Dawn,Anatole Litvak
2,1951,BEST MOTION PICTURE,Decision before Dawn,Frank McCarthy
3,1951,BEST MOTION PICTURE,A Place in the Sun,George Stevens
4,1951,BEST MOTION PICTURE,Quo Vadis,Sam Zimbalist
...,...,...,...,...
783,2021,BEST PICTURE,The Power of the Dog,Emile Sherman
784,2021,BEST PICTURE,The Power of the Dog,Iain Canning
785,2021,BEST PICTURE,The Power of the Dog,Roger Frappier
786,2021,BEST PICTURE,West Side Story,Steven Spielberg
